<a href="https://colab.research.google.com/github/LuizJoca/shoreline_analisys_protocol/blob/main/Extra%C3%A7%C3%A3o_Semi_Autom%C3%A1tica_de_Linha_de_Costa_ptbr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color = khaki>**Extração Semi-Automática de Linha de Costa**</font>

## <font color = khaki>**Preparação de Ambiente**</font>

### <font color = darkorange>Importando as bibliotecas</font>

In [1]:
# Importação das Bibliotecas

import cv2
import numpy as np

from osgeo import osr
from osgeo import gdal
from osgeo import gdal_array

from skimage import io
from skimage import img_as_ubyte
from skimage.filters import median
from skimage.morphology import disk

from matplotlib import pyplot as plt

### <font color = darkorange>Importação das Imagens Pré-Tratadas</font>

In [ ]:
img_flaash =

In [ ]:
ds = gdal.Open(img_flaash)
gt = ds.GetGeoTransform()
proj = dj.GetProjection()

In [ ]:
mndwi =  ds.GetRasterBand(1)

In [ ]:
mndwi_array = mndwi.ReadAsArray()

In [ ]:
plt.figure()
plt.imshow(mndwi_array)
plt.colorbar()

## <font color = khaki>**Aplicação do Filtro de Mediana**</font>

In [ ]:
mndwi_median = median(mndwi_array, disk(3), mode = 'constant', cval = 0.0)

In [ ]:
plt.figure()
plt.imshow(mndwi_median)
plt.colorbar()

## <font color = khaki>**Conversão da Imagem para 8bit**</font>

In [ ]:
mndwi_8bit = img_as_ubyte(mndwi_median)

In [ ]:
mndwi_8bit.dtype

## <font color = khaki>**Aplicação do Threshold**</font>

In [ ]:
ret_mndwi, th_mndwi = cv2.threshold(mndwi_8bit, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

In [ ]:
plt.figure()
plt.imshow(th_mndwi)
plt.colorbar()

## <font color = khaki>**Exportação do Arquivo Binarizado**</font>

In [ ]:
driver = gdal.GetDriverByName('ENVI')
driver.Register()
outds = driver.Create('mndwi_threshold.dat',
                      xsize = mndwi_threshold.shape[1],
                      ysize = mndwi_threshold.shape[0],
                      bands = 1,
                      eType = gdal.GDT_Byte)
outds.SetGeoTransform(gt)
outds.SetProjection(proj)
outband = outds.GetRasterBand(1)
outband.WriteArray(mndwi_threshold)
outband.SetNoDataValue(0)
outband.FlushCache()

outband = None
outds = None